# Visualization Workflow

## Install packages

In [1]:
print("Installing packages (this could take awhile)...")

import sys
!$sys.executable -m pip install  -r ../requirements.txt --quiet --exists-action i
print("Done!")


Installing packages (this could take awhile)...
Done!


In [2]:
import os, logging
import pathlib
import typing
from typing import Optional, Dict
import time
import warnings
module_path = os.path.abspath(os.path.join('../utils'))
if module_path not in sys.path:
    sys.path.append(module_path)
from func import *

## Inputs

In [3]:
inpDir = pathlib.Path('/opt/shared/notebooks/eastman/fulldata/polus-data/polus/images/Eastman2021Infectivity/standard/intensity')
outDir = pathlib.Path('/opt/shared/notebooks/eastman/tmp5')
platesNum=200
dryrun=False


### Running All functions Sequentially

In [5]:
logging.basicConfig(
    format='%(asctime)s - %(name)-8s - %(levelname)-8s - %(message)s',
    datefmt='%d-%b-%y %H:%M:%S',
)
logger = logging.getLogger('Visualization Workflow')
logger.setLevel(logging.INFO)

warnings.filterwarnings('ignore')

@my_timer 
def visualizations_plugins(inpDir:pathlib.Path,
                    outDir: pathlib.Path,
                    platesNum:int,
                    dryrun:bool=True         
                   ):
        
    plates = [str(i).zfill(3) for i in range(1, platesNum+1)][:2]

    assert platesNum != 0, "Number of Processing Plates should be greater than zero"

    filePatterns = ['p'+ pl +  '_x{x+}_y{y+}_wx{t}_wy{p}_c{c}.ome.tif'  for pl in plates]

    # assert len(filePatterns) == platesNum, "Number of filepattern should be equivalent to number of plates"

    for plate, filePattern in zip(plates, filePatterns):

        logger.info(f'Workflow runs for plate:{plate} & filePattern:{filePattern}')

        logger.info(f'Running 1st plugin: basic_flatfield on plate: p{plate}')


        bpath = basic_flatfield(inpDir,
                                   outDir,
                                   filePattern,
                                   plate,
                                   dryrun=dryrun)
        

        logger.info(f'Running 2nd plugin: apply_flatfield on plate: p{plate}')

        corrDir = apply_flatfield(inpDir,
                           outDir,
                           ffDir=bpath,
                           filePattern=filePattern,
                           plate=plate,
                           dryrun=dryrun)

        logger.info(f'Running 3rd plugin: montage on plate: p{plate}')

        mpath = montage(inpDir=corrDir,
                           outDir=outDir,
                           filePattern=filePattern,
                           plate=plate,
                           dryrun=dryrun) 

        logger.info(f'Running 4th plugin: stitchvector on plate: p{plate}')

        stitchpath = stichingvector(inpDir=mpath,
                            outDir=outDir,
                            plate=plate,
                            dryrun=dryrun)


        logger.info(f'Running 5th plugin: assemble on plate: p{plate}')


        asspath = assemble(inpDir=corrDir,
                    outDir=outDir,
                    stitchPath=stitchpath,
                    filePattern=filePattern,
                    plate=plate,
                    dryrun=dryrun)
        
        logger.info(f'Running 6th plugin: precomputeslide on plate: p{plate}')
        
        
        # outpre = pathlib.Path('/home/jovyan/shared/data/file-server/pyramids/eastmannew/zarrpyramids_intensity')
        precompath = precompute_slide(inpDir=asspath,
                       outDir=outDir,
                       plate=plate,
                       imageType='image',
                       pyramidType="Zarr",
                       dryrun=dryrun)

    return


visualizations_plugins(inpDir,
                    outDir,
                    platesNum,
                    dryrun=dryrun              
                   )    

08-Sep-22 16:32:03 - basicpy.tools.dct_tools - WARNING  - the value of the environment variable BASIC_DCT_BACKEND is not in ["JAX","SCIPY"]
08-Sep-22 16:32:03 - main     - INFO     - Parsing arguments...
08-Sep-22 16:32:03 - main     - INFO     - input_dir = /opt/shared/notebooks/eastman/fulldata/polus-data/polus/images/Eastman2021Infectivity/standard/intensity
08-Sep-22 16:32:03 - main     - INFO     - get_darkfield = True
08-Sep-22 16:32:03 - main     - INFO     - get_photobleach = False
08-Sep-22 16:32:03 - main     - INFO     - file_pattern = p001_x{x+}_y{y+}_wx{t}_wy{p}_c{c}.ome.tif
08-Sep-22 16:32:03 - main     - INFO     - output_dir = /opt/shared/notebooks/eastman/tmp5/basic_flatfield/p001_basic_flatfield/images
08-Sep-22 16:32:09 - main     - INFO     - Running on 4 processes.
08-Sep-22 16:32:16 - proc 1   - INFO     - p001_x(01-24)_y(01-16)_wx(1-3)_wy(1-3)_c1.ome.tif: Loading images...
08-Sep-22 16:32:16 - proc 2   - INFO     - p001_x(01-24)_y(01-16)_wx(1-3)_wy(1-3)_c2.ome.ti